In [ ]:
import speech_recognition as sr
import numpy as np
import matplotlib.pyplot as plt
import time
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import pyaudio
import audioop

# Download VADER lexicon if not already downloaded
nltk.download('vader_lexicon')

class LiveSentimentAnalyzer:
    def __init__(self):
        self.recognizer = sr.Recognizer()
        self.analyzer = SentimentIntensityAnalyzer()
        self.timestamps = []
        self.sentiment_values = []
        self.loudness_values = []
        self.start_time = time.time()
        
    def setup_visualization(self):
        self.fig, (self.ax1, self.ax2) = plt.subplots(2, 1, figsize=(10, 8))
        self.fig.suptitle('Real-Time Voice Sentiment Analysis')
        
        # Sentiment plot
        self.ax1.set_xlabel('Time (s)')
        self.ax1.set_ylabel('Sentiment Score')
        self.ax1.set_ylim(-1, 1)
        self.ax1.grid(True)
        
        # Loudness plot
        self.ax2.set_xlabel('Time (s)')
        self.ax2.set_ylabel('Loudness (RMS)')
        self.ax2.grid(True)
    
    def capture_and_analyze(self):
        with sr.Microphone() as source:
            self.recognizer.adjust_for_ambient_noise(source)
            print("Listening...")
            audio = self.recognizer.listen(source)
            
            # Get raw audio data for loudness analysis
            raw_audio = audio.get_raw_data()
            # Calculate RMS as a measure of loudness
            loudness = audioop.rms(raw_audio, 2)  # 2 bytes per sample
            
            try:
                text = self.recognizer.recognize_google(audio)
                sentiment_scores = self.analyzer.polarity_scores(text)
                
                if sentiment_scores['compound'] >= 0.05:
                    sentiment = "Positive"
                elif sentiment_scores['compound'] <= -0.05:
                    sentiment = "Negative"
                else:
                    sentiment = "Neutral"
                
                current_time = time.time() - self.start_time
                self.timestamps.append(current_time)
                self.sentiment_values.append(sentiment_scores['compound'])
                self.loudness_values.append(loudness)
                
                # Adjust sentiment based on loudness
                adjusted_sentiment = self.adjust_sentiment_with_loudness(sentiment_scores['compound'], loudness)
                
                return text, sentiment, adjusted_sentiment, sentiment_scores, loudness
            except:
                return "", "Unknown", "Unknown", {"compound": 0}, 0
    
    def adjust_sentiment_with_loudness(self, sentiment_score, loudness):
        # Normalize loudness to a scale of 0-1
        max_loudness = 32767  # Maximum possible RMS value for 16-bit audio
        normalized_loudness = min(1.0, loudness / max_loudness)
        
        # If loudness is high and sentiment is already leaning in a direction, amplify it
        if normalized_loudness > 0.6:  # Threshold for "loud" speech
            if sentiment_score > 0:
                # Amplify positive sentiment for loud, positive speech
                adjusted_score = min(1.0, sentiment_score * (1 + normalized_loudness * 0.5))
                return adjusted_score
            elif sentiment_score < 0:
                # Amplify negative sentiment for loud, negative speech
                adjusted_score = max(-1.0, sentiment_score * (1 + normalized_loudness * 0.5))
                return adjusted_score
        
        return sentiment_score
    
    def update_visualization(self, sentiment, score, loudness):
        # Clear previous plots
        self.ax1.clear()
        self.ax2.clear()
        
        # Update sentiment plot
        self.ax1.plot(self.timestamps, self.sentiment_values, 'b-')
        self.ax1.set_title(f'Current Sentiment: {sentiment} (Score: {score:.2f})')
        self.ax1.set_xlabel('Time (s)')
        self.ax1.set_ylabel('Sentiment Score')
        self.ax1.set_ylim(-1, 1)
        self.ax1.grid(True)
        
        # Update loudness plot
        self.ax2.plot(self.timestamps, self.loudness_values, 'r-')
        self.ax2.set_title(f'Current Loudness: {loudness}')
        self.ax2.set_xlabel('Time (s)')
        self.ax2.set_ylabel('Loudness (RMS)')
        self.ax2.grid(True)
        
        plt.tight_layout()
        plt.subplots_adjust(top=0.9)
        plt.pause(0.01)
    
    def extract_audio_features(self, audio):
        # Convert audio to numpy array
        audio_data = np.array(audio.get_array_of_samples())
        
        # Extract features
        features = {
            "volume": np.mean(np.abs(audio_data)),
            "zero_crossings": np.sum(np.diff(np.signbit(audio_data))),
            "energy": np.sum(audio_data**2) / len(audio_data),
            "max_amplitude": np.max(np.abs(audio_data))
        }
        
        return features
    
    def run(self):
        self.setup_visualization()
        
        while True:
            text, sentiment, adjusted_sentiment, scores, loudness = self.capture_and_analyze()
            
            if text:
                self.update_visualization(sentiment, scores["compound"], loudness)
                
                print(f"Text: {text}")
                print(f"Raw Sentiment: {sentiment} (Score: {scores['compound']:.2f})")
                print(f"Adjusted Sentiment (with loudness): {adjusted_sentiment:.2f}")
                print(f"Loudness: {loudness}")
                print("-" * 50)

if __name__ == "__main__":
    analyzer = LiveSentimentAnalyzer()
    analyzer.run()
